In [1]:
# Warning: Execute this cell only once for the kernel. Reset the kernel if you need changes.
import sys
sys.path.append("../Python")
import ROOT as R
import time
import numpy as np
# Turn jsroot off if you want to make a pdf from this file.
%jsroot on
from root_helpers import SetStyle
from root_helpers import fancy_plot
from root_helpers import print_mc_particle_tree
from root_helpers import print_daughters
# R.EnableImplicitMT()
import os
recompile = True
try:
    if os.path.getmtime('../Python/Utility_Functions_C.so') - os.path.getmtime('../Python/Utility_Functions.C') > 0:
        recompile = False
        print("Recompile is not needed")
    else:
        print("Recompiling: ")
except:
    print("Recompile needed, file not found.")
if recompile:
    R.gROOT.LoadMacro("../Python/Utility_Functions.C++")
else:
    R.gSystem.Load("../Python/Utility_Functions_C.so")
R.Utility_Functions()

Welcome to JupyROOT 6.29/01
Recompile is not needed


'Utility Functions V1.0.4 \n'

In [2]:
ch = R.TChain("MiniDST")
energy="1.0"
particle="e-"
if particle == "e+":
    particle_s = "e\+"
else:
    particle_s = particle
ch.Add("/data/HPS/data/physrun2021/MC/hpsForward_"+particle_s+"_"+energy+"GeV_z0.0_*_SLIC-v06-00-01_QGSP_BERT_HPS-v2019-3pt7GeV_recon.root")
# ch.Add("/data/HPS/data/physrun2021/pass0/minidst/hps_0147*.root")
print(f"Number of events loaded: {ch.GetEntries()/1e6:7.3f}M")
df = R.RDataFrame(ch)

Number of events loaded:   0.687M


In [3]:
dfx = df.Define("n_kf_track","int n=0; for(int i=0;i<track_type.size();++i){if(track_type[i]==1) n+=1;}; return n") \
    .Filter("n_kf_track==1").Filter("ecal_cluster_energy.size()==1") \
    .Define("trk_idx","for(int i=0;i<track_type.size();++i){if(track_type[i]==1) return(i);} return(-1);") \
    .Define("diff_posx_ecal","ecal_cluster_x[0] - track_x_at_ecal[trk_idx]") \
    .Define("diff_posy_ecal","ecal_cluster_y[0] - track_y_at_ecal[trk_idx]") \
    .Define("diff_E_p","part_energy[0] - get_vec_abs(part_px,part_py,part_pz)[0]") \
    .Define("diff_p_kf_gbl","get_vec_abs(part_px,part_py,part_pz)[0] - get_vec_abs(part_px,part_py,part_pz)[1]") \
    .Define("mc_part_primary_z","find_end_z_of_primary_mc_part(mc_part_z, mc_part_end_z)") \
    .Define("mc_part_daughters_z","find_average_end_of_daughters_of_primary_mc_part(mc_part_z, mc_part_end_z, mc_part_daughters)")
cnt  = df.Count()
cntx = dfx.Count()

In [8]:
hx_diff_posx_ecal=dfx.Histo1D(("hx_diff_posx","Diff Ecal_x - Track_x",1000,-20.,20.),"diff_posx_ecal")
hx_diff_posx_ecal_top=dfx.Filter("ecal_cluster_y[0]>0").Histo1D(("hx_diff_posx_top","Diff Ecal_x - Track_x Top",1000,-20.,20.),"diff_posx_ecal")
hx_diff_posx_ecal_bot=dfx.Filter("ecal_cluster_y[0]<0").Histo1D(("hx_diff_posx_bot","Diff Ecal_x - Track_x Bot",1000,-20.,20.),"diff_posx_ecal")
hx_diff_posx_ecal_L=dfx.Filter("ecal_cluster_x[0]<0").Histo1D(("hx_diff_posx_L","Diff Ecal_x - Track_x L",1000,-20.,20.),"diff_posx_ecal")
hx_diff_posx_ecal_R=dfx.Filter("ecal_cluster_x[0]>0").Histo1D(("hx_diff_posx_R","Diff Ecal_x - Track_x R",1000,-20.,20.),"diff_posx_ecal")
hx_diff_posy_ecal=dfx.Histo1D(("hx_diff_posy","Diff Ecal_y - Track_y",1000,-20.,20.),"diff_posy_ecal")
hx_diff_posy_ecal_top=dfx.Filter("ecal_cluster_y[0]>0").Histo1D(("hx_diff_posy_top","Diff Ecal_y - Track_y Top",1000,-20.,20.),"diff_posy_ecal")
hx_diff_posy_ecal_bot=dfx.Filter("ecal_cluster_y[0]<0").Histo1D(("hx_diff_posy_bot","Diff Ecal_y - Track_y Bot",1000,-20.,20.),"diff_posy_ecal")
hx_diff_posy_ecal_L=dfx.Filter("ecal_cluster_x[0]<0").Histo1D(("hx_diff_posy_L","Diff Ecal_y - Track_y L",1000,-20.,20.),"diff_posy_ecal")
hx_diff_posy_ecal_R=dfx.Filter("ecal_cluster_x[0]>0").Histo1D(("hx_diff_posy_R","Diff Ecal_y - Track_y R",1000,-20.,20.),"diff_posy_ecal")
hx_diff_e_p = dfx.Histo1D(("hx_diff_e_p", "Diff Energy - Momentum", 1000, -3., 1.), "diff_E_p")
hx_diff_e_p_top = dfx.Filter("ecal_cluster_y[0]>0").Histo1D(("hx_diff_e_p_top", "Diff Energy - Momentum Top", 1000, -3., 1.), "diff_E_p")
hx_diff_e_p_bot = dfx.Filter("ecal_cluster_y[0]<0").Histo1D(("hx_diff_e_p_bot", "Diff Energy - Momentum Bot", 1000, -3., 1.), "diff_E_p")
hx_diff_e_p_L = dfx.Filter("ecal_cluster_x[0]<0").Histo1D(("hx_diff_e_p_L", "Diff Energy - Momentum Left", 1000, -3., 1.), "diff_E_p")
hx_diff_e_p_R = dfx.Filter("ecal_cluster_x[0]>0").Histo1D(("hx_diff_e_p_R", "Diff Energy - Momentum Right", 1000, -3., 1.), "diff_E_p")
hx_diff_p_kf_gbl = dfx.Histo1D(("hx_diff_p_kf_gbl","Diff P_kf - P_gbl",1000,-0.2,0.2),"diff_p_kf_gbl")
hx_diff_p_kf_gbl_top = dfx.Filter("ecal_cluster_y[0]>0").Histo1D(("hx_diff_p_kf_gbl_top","Diff P_kf - P_gbl",1000,-0.2,0.2),"diff_p_kf_gbl")
hx_diff_p_kf_gbl_bot = dfx.Filter("ecal_cluster_y[0]<0").Histo1D(("hx_diff_p_kf_gbl_bot","Diff P_kf - P_gbl",1000,-0.2,0.2),"diff_p_kf_gbl")
hx_diff_p_kf_gbl_L = dfx.Filter("ecal_cluster_x[0]<0").Histo1D(("hx_diff_p_kf_gbl_L","Diff P_kf - P_gbl",1000,-0.2,0.2),"diff_p_kf_gbl")
hx_diff_p_kf_gbl_R = dfx.Filter("ecal_cluster_x[0]>0").Histo1D(("hx_diff_p_kf_gbl_R","Diff P_kf - P_gbl",1000,-0.2,0.2),"diff_p_kf_gbl")

hx_mc_part_primary_z = dfx.Histo1D(("hx_mc_part_primary_z","End of track for primary MC particle.",1000,-0.1,1700.),"mc_part_primary_z")
hx_mc_part_ave_daughter_z = dfx.Histo1D(("hx_mc_part_ave_daughter_z","Average end of track for primary MC particle daughters.",1000,-0.1,1700.),"mc_part_daughters_z")


In [5]:
cc4 = R.TCanvas("cc4","Canvas",1200,600)
cc4.Divide(2,2)
p1=cc4.cd(1)
#p1.SetLogy()
hx_diff_posx_ecal.Draw()
hx_diff_posx_ecal_top.SetLineColor(R.kGreen)
hx_diff_posx_ecal_top.Draw("same")
hx_diff_posx_ecal_bot.SetLineColor(R.kRed)
hx_diff_posx_ecal_bot.Draw("same")
hx_diff_posx_ecal_L.SetLineColor(R.kMagenta)
hx_diff_posx_ecal_L.Draw("same")
hx_diff_posx_ecal_R.SetLineColor(R.kCyan)
hx_diff_posx_ecal_R.Draw("same")
p2=cc4.cd(2)
#p2.SetLogy()
hx_diff_posy_ecal.Draw()
hx_diff_posy_ecal_top.SetLineColor(R.kGreen)
hx_diff_posy_ecal_top.Draw("same")
hx_diff_posy_ecal_bot.SetLineColor(R.kRed)
hx_diff_posy_ecal_bot.Draw("same")
hx_diff_posy_ecal_L.SetLineColor(R.kMagenta)
hx_diff_posy_ecal_L.Draw("same")
hx_diff_posy_ecal_R.SetLineColor(R.kCyan)
hx_diff_posy_ecal_R.Draw("same")
p3=cc4.cd(3)
# p3.SetLogy()
hx_diff_e_p.Draw()
hx_diff_e_p_top.SetLineColor(R.kGreen)
hx_diff_e_p_top.Draw("same")
hx_diff_e_p_bot.SetLineColor(R.kRed)
hx_diff_e_p_bot.Draw("same")
hx_diff_e_p_L.SetLineColor(R.kMagenta)
hx_diff_e_p_L.Draw("same")
hx_diff_e_p_R.SetLineColor(R.kCyan)
hx_diff_e_p_R.Draw("same")
p4=cc4.cd(4)
# p4.SetLogy()
hx_diff_p_kf_gbl.Draw()
hx_diff_p_kf_gbl_top.SetLineColor(R.kGreen)
hx_diff_p_kf_gbl_top.Draw("same")
hx_diff_p_kf_gbl_bot.SetLineColor(R.kRed)
hx_diff_p_kf_gbl_bot.Draw("same")
hx_diff_p_kf_gbl_L.SetLineColor(R.kMagenta)
hx_diff_p_kf_gbl_L.Draw("same")
hx_diff_p_kf_gbl_R.SetLineColor(R.kCyan)
hx_diff_p_kf_gbl_R.Draw("same")

# h_trk_type.Draw()
cc4.cd(0)
leg1 = R.TLegend(0.83,0.8,1.,1.)
leg1.AddEntry(hx_diff_posx_ecal.GetValue(), "All data with one cluster")
leg1.AddEntry(hx_diff_posx_ecal_top.GetValue(),"Top only")
leg1.AddEntry(hx_diff_posx_ecal_bot.GetValue(),"Bottom only")
leg1.AddEntry(hx_diff_posx_ecal_L.GetValue(),"Left Only x<0")
leg1.AddEntry(hx_diff_posx_ecal_R.GetValue(),"Right Only x>0")
leg1.Draw()

cc4.Draw()
cc4.SaveAs("MC_"+particle+"_"+energy+"GeV_compare.pdf")

Info in <TCanvas::Print>: pdf file MC_e-_1.0GeV_compare.pdf has been created


In [10]:
cc5 = R.TCanvas("cc5","Canvas",800,600)
hx_mc_part_ave_daughter_z.SetStats(0)
hx_mc_part_ave_daughter_z.SetLineColor(R.kRed)
hx_mc_part_ave_daughter_z.Draw()

hx_mc_part_primary_z.SetLineColor(R.kBlue)
hx_mc_part_primary_z.Draw("same")
cc5.Draw()

Warning in <TCanvas::Constructor>: Deleting canvas with same name: cc5
